# OLS and lasso for gender wage gap inference

In the previous lab, we analyzed data from the March Supplement of the U.S. Current Population Survey (2015) and answered the question of how to use job-relevant characteristics, such as education and experience, to best predict wages. Now, we focus on the following inference question:

What is the difference in predicted wages between men and women with the same job-relevant characteristics?

Thus, we analyze if there is a difference in the payment of men and women (*gender wage gap*). The gender wage gap may partly reflect *discrimination* against women in the labor market or may partly reflect a *selection effect*, namely that women are relatively more likely to take on occupations that pay somewhat less (for example, school teaching).

To investigate the gender wage gap, we consider the following log-linear regression model

$$
\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 D  + \beta_2' W + \epsilon,
\end{align}
$$

where $Y$ is hourly wage, $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the
$W$'s are a vector of worker characteristics explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of men and women.

## Data analysis

We consider the same subsample of the U.S. Current Population Survey (2015) as in the previous lab. Let us load the data set.

In [1]:
install.packages("librarian", quiet = T)
librarian::shelf(tidyverse, sandwich, hdm, quiet = T)
data <- read_csv("https://github.com/d2cml-ai/14.388_R/raw/main/Data/wage2015_subsample_inference.csv"
        , show_col_types = F)
dim(data)
attach(data)

[1] 5150   21

To start our (causal) analysis, we compare the sample means given gender:

In [2]:
variables <- c("lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1")

Z <- data |> select(all_of(variables))

data_female <- data |> filter(sex == 1)
data_male <- data |> filter(sex == 0)

Z_mean = Z |>
  mutate(sex = 3) |> # ALL 
  bind_rows(Z) |> # Sex
  group_by(sex) |>
  summarise(across(where(is.numeric), mean)) |>
  ungroup() |>
  mutate(sex = case_when(sex == 1 ~ "Female", sex == 0 ~ "Male", T ~ "All"))

colnames(Z_mean) <- c("Sex","Log Wage","Less then High School","High School Graduate","Some College","College Graduate","Advanced Degree", "Northeast","Midwest","South","West","Experience")
Z_mean |>
  pivot_longer(!Sex, names_to = "Variable") |>
  pivot_wider(names_from = Sex, values_from = value)

Variable,Male,Female,All
<chr>,<dbl>,<dbl>,<dbl>
Log Wage,2.98782963,2.94948490,2.97078670
Less then High School,0.03180706,0.01266929,0.02330097
High School Graduate,0.29430269,0.18086501,0.24388350
Some College,0.27333100,0.28396680,0.27805825
College Graduate,0.29395316,0.34731324,0.31766990
Advanced Degree,0.10660608,0.17518567,0.13708738
Northeast,0.22195037,0.23503713,0.22776699
Midwest,0.25900035,0.26037571,0.25961165
South,0.29814750,0.29445173,0.29650485


In particular, the table above shows that the difference in average *logwage* between men and women is equal to $0.038$

In [3]:
mean(data_female$lwage)-mean(data_male$lwage)

[1] -0.03834473

Thus, the unconditional gender wage gap is about $3,8$\% for the group of never married workers (women get paid less on average in our sample). We also observe that never married working women are relatively more educated than working men and have lower working experience.

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

$$
\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}
$$

We verify this by running an ols regression in R.

In [4]:
nocontrol_fit <- lm(lwage ~ sex, data = Z)
nocontrol_est <- summary(nocontrol_fit)$coef["sex",1]
HCV_coefs <- vcovHC(nocontrol_fit, type = 'HC'); # HC - "heteroskedasticity cosistent"
nocontrol_se <- sqrt(diag(HCV_coefs))[2] # Estimated std errors

# print unconditional effect of gender and the corresponding standard error
cat ("The estimated coefficient on the dummy for gender is",nocontrol_est,"\nand the corresponding robust standard error is", nocontrol_se) 

The estimated coefficient on the dummy for gender is -0.03834473 
and the corresponding robust standard error is 0.01590194

Note that the standard error is computed with the *R* package *sandwich* to be robust to heteroskedasticity. 


Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

$$
\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}
$$

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

In [5]:
# ols regression with controls

flex <- lwage ~ sex + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg+ occ2 + ind2 + mw + so + we)

#   Note that ()*() operation in formula objects in R creates a formula of the sort:
#  (exp1+exp2+exp3+exp4)+ (shs+hsg+scl+clg+occ2+ind2+mw+so+we) + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
#  This is not intuitive at all, but that's what it does.

control_fit <- lm(flex, data = data)
control_est <- summary(control_fit)$coef[2,1]

summary(control_fit)

cat("Coefficient for OLS with controls", control_est)

HCV_coefs <- vcovHC(control_fit, type = 'HC');
control_se <- sqrt(diag(HCV_coefs))[2] # Estimated std errors


Call:
lm(formula = flex, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1282 -0.3065 -0.0151  0.2945  3.5341 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.5459818  0.1293099  27.422  < 2e-16 ***
sex         -0.1024693  0.0146380  -7.000 2.89e-12 ***
exp1         0.0395328  0.0481220   0.822  0.41139    
exp2        -0.0470984  0.5325375  -0.088  0.92953    
exp3        -0.0771429  0.2154298  -0.358  0.72029    
exp4         0.0192883  0.0284494   0.678  0.49781    
shs         -0.6909668  0.8988057  -0.769  0.44207    
hsg         -0.5816131  0.1944598  -2.991  0.00279 ** 
scl         -0.3297645  0.1234560  -2.671  0.00758 ** 
clg         -0.0873795  0.0668027  -1.308  0.19092    
occ2        -0.0266617  0.0050951  -5.233 1.74e-07 ***
ind2        -0.0161333  0.0062181  -2.595  0.00950 ** 
mw           0.1079623  0.0834048   1.294  0.19557    
so           0.0385942  0.0747084   0.517  0.60546    
we          -0.0035042  0.0

Coefficient for OLS with controls -0.1024693

The estimated regression coefficient $\beta_1\approx-0.0696$ measures how our linear prediction of wage changes if we set the gender variable $D$ from 0 to 1, holding the controls $W$ fixed.
We can call this the *predictive effect* (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size $4$\% for women increases to about $7$\% after controlling for worker characteristics.  


Next, we use the Frisch-Waugh-Lovell (FWL) theorem from lecture, partialling-out the linear effect of the controls via ols.

In [6]:
# Partialling-out using ols

# models
flex_y <- lwage ~ (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl +clg + occ2+ ind2 + mw + so + we) # model for Y
flex_d <- sex ~ (exp1 + exp2 + exp3 + exp4)*(shs + hsg + scl + clg + occ2 + ind2 + mw + so + we) # model for D

# partialling-out the linear effect of W from Y
t_Y <- lm(flex_y, data=data)$res
# partialling-out the linear effect of W from D
t_D <- lm(flex_d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partial_fit <- lm(t_Y ~ t_D)
partial_est <- summary(partial_fit)$coef[2,1]

cat("Coefficient for D via partialling-out", partial_est)

# standard error
HCV_coefs <- vcovHC(partial_fit, type = 'HC')
partial_se <- sqrt(diag(HCV_coefs))[2]

# confidence interval
confint(partial_fit)[2,]

Coefficient for D via partialling-out -0.1024693

2.5 %      97.5 % 
-0.13102916 -0.07390946

Again, the estimated coefficient measures the linear predictive effect (PE) of $D$ on $Y$ after taking out the linear effect of $W$ on both of these variables. This coefficient is numerically equivalent to the estimated coefficient from the ols regression with controls, confirming the FWL theorem.

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 

In the following, we illustrate the partialling-out approach using lasso instead of ols. 

In [7]:
# Partialling-out using lasso

library(hdm)

# models
flex_y <- lwage ~ (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + occ2 + ind2 + mw + so + we) # model for Y
flex_d <- sex ~ (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + occ2 + ind2 + mw + so + we) # model for D

# partialling-out the linear effect of W from Y
t_Y <- rlasso(flex_y, data = data)$res
# partialling-out the linear effect of W from D
t_D <- rlasso(flex_d, data = data)$res

# regression of Y on D after partialling-out the effect of W
partial_lasso_fit <- lm(t_Y ~ t_D)
partial_lasso_est <- summary(partial_lasso_fit)$coef[2,1]

cat("Coefficient for D via partialling-out using lasso", partial_lasso_est)

# standard error
HCV_coefs <- vcovHC(partial_lasso_fit, type = 'HC')
partial_lasso_se <- sqrt(diag(HCV_coefs))[2]

Coefficient for D via partialling-out using lasso -0.1036713

Using lasso for partialling-out here provides similar results as using ols.

Next, we summarize the results.

In [8]:
table<- matrix(0, 4, 2)
table[1,1]<- nocontrol_est  
table[1,2]<- nocontrol_se   
table[2,1]<- control_est
table[2,2]<- control_se    
table[3,1]<- partial_est  
table[3,2]<- partial_se  
table[4,1]<-  partial_lasso_est
table[4,2]<- partial_lasso_se 
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("Without controls", "full reg", "partial reg", "partial reg via lasso")	
table

,Estimate,Std. Error
Without controls,-0.03834473,0.01590194
full reg,-0.10246931,0.01458860
partial reg,-0.10246931,0.01458860
partial reg via lasso,-0.10367131,0.01475760


It it worth noticing that controlling for worker characteristics increases the gender wage gap from less than 4\% to 7\%. The controls we used in our analysis include 5 educational attainment indicators (less than high school graduates, high school graduates, some college, college graduate, and advanced degree), 4 region indicators (midwest, south, west, and northeast);  a quartic term (first, second, third, and fourth power) in experience and 22 occupation and 23 industry indicators.

Keep in mind that the predictive effect (PE) does not only measures discrimination (causal effect of being female), it also may reflect
selection effects of unobserved differences in covariates between men and women in our sample.


Next we try an "extra" flexible model, where we take interactions of all controls, giving us about 1000 controls.

In [9]:
# extra flexible model

extraflex <- lwage ~ sex + (exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we)^2

control_fit <- lm(extraflex, data=data)
summary(control_fit)
control_est <- summary(control_fit)$coef[2,1]

cat("Number of Extra-Flex Controls", length(control_fit$coef) - 1, "\n")

cat("Coefficient for OLS with extra flex controls", control_est)

HCV_coefs <- vcovHC(control_fit, type = 'HC');

n = length(wage); p =length(control_fit$coef);

control_se <- sqrt(diag(HCV_coefs))[2] * sqrt( n / (n - p)) # Estimated std errors

# This is a crude adjustment for the effect of dimensionality on OLS standard errors, 
# motivated by Cattaneo, Jannson, and Newey (2018). For a more correct approach, we 
# would implement the approach of Cattaneo, Jannson, and Newey (2018)'s procedure.


Call:
lm(formula = extraflex, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.1158 -0.3041 -0.0173  0.2893  3.5892 

Coefficients: (12 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.6297982  0.1552865  23.375  < 2e-16 ***
sex         -0.0967413  0.0146627  -6.598 4.60e-11 ***
exp1         0.0094395  0.0613320   0.154  0.87769    
exp2         0.7798590  1.3068326   0.597  0.55070    
exp3        -0.9937088  1.5435940  -0.644  0.51976    
exp4         0.5284032  0.9687560   0.545  0.58547    
shs          0.4280438  1.0441100   0.410  0.68185    
hsg         -0.5051460  0.2742205  -1.842  0.06552 .  
scl         -0.2951033  0.1897037  -1.556  0.11987    
clg         -0.1203554  0.1176922  -1.023  0.30653    
occ2        -0.0331114  0.0071125  -4.655 3.32e-06 ***
ind2        -0.0203271  0.0076790  -2.647  0.00814 ** 
mw           0.1000058  0.1185443   0.844  0.39892    
so           0.0590348  0.1071524  

Number of Extra-Flex Controls 92 


Coefficient for OLS with extra flex controls -0.09674132

In [10]:
library(hdm)

# models
extraflex_y <- lwage ~ (exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we)^2 # model for Y
extraflex_d <- sex ~ (exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we)^2 # model for D

# partialling-out the linear effect of W from Y
t_Y <- rlasso(extraflex_y, data=data)$res
# partialling-out the linear effect of W from D
t_D <- rlasso(extraflex_d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partial_lasso_fit <- lm(t_Y~t_D)
partial_lasso_est <- summary(partial_lasso_fit)$coef[2,1]

cat("Coefficient for D via partialling-out using lasso", partial_lasso_est)

# standard error
HCV_coefs <- vcovHC(partial_lasso_fit, type = 'HC')
partial_lasso_se <- sqrt(diag(HCV_coefs))[2]

Coefficient for D via partialling-out using lasso -0.09916518

In [11]:
table<- matrix(0, 2, 2)
table[1,1]<- control_est
table[1,2]<- control_se    
table[2,1]<-  partial_lasso_est
table[2,2]<- partial_lasso_se 
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("full reg","partial reg via lasso")	
table

,Estimate,Std. Error
full reg,-0.09674132,0.01476431
partial reg via lasso,-0.09916518,0.01481612


In this case p/n = 20%, that is $p/n$ is no longer small and we start seeing the differences between
unregularized partialling out and regularized partialling out with lasso (double lasso).  The results based on 
double lasso have rigorous guarantees in this non-small p/n regime under approximate sparsity. The results based on OLS still
have guarantees in p/n< 1 regime under assumptions laid out in Cattaneo, Newey, and Jansson (2018), without approximate
sparsity, although other regularity conditions are needed.
